# Function 4

## Function Description
Address the challenge of optimally placing products across warehouses for a business with high online sales, where accurate calculations are costly and only feasible biweekly. To speed up decision-making, an ML model approximates these results within hours. The model has four hyperparameters to tune, and its output reflects the difference from the expensive baseline. Because the system is dynamic and full of local optima, it requires careful tuning and robust validation to find reliable, near-optimal solutions.

## Libraries

In [1]:
import pandas as pd
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel

## Data

In [2]:
# Initialize the dataset
df_init = pd.DataFrame({
    "x1": [0.896981054, 0.889356396, 0.250946243, 0.346962061, 0.124871181,
           0.801302707, 0.247708262, 0.746702242, 0.400665027, 0.626070596,
           0.957135293, 0.732812426, 0.655115479, 0.219734429, 0.48859419,
           0.167130486, 0.216911188, 0.387487837, 0.985621893, 0.037824829,
           0.683486385, 0.170347305, 0.859656919, 0.282138368, 0.326075785,
           0.948389362, 0.66495539, 0.577765614, 0.738613014, 0.854810797],

    "x2": [0.72562797, 0.499587855, 0.033693131, 0.0062504, 0.129770193,
           0.500231094, 0.060445427, 0.757091504, 0.072574251, 0.586751259,
           0.597644383, 0.145249979, 0.072391827, 0.832031335, 0.211965096,
           0.876554558, 0.166085829, 0.804532258, 0.666932679, 0.664853346,
           0.902770103, 0.756959083, 0.919592322, 0.505986912, 0.472366904,
           0.894513008, 0.046566277, 0.428771742, 0.482102634, 0.49396462],

    "x3": [0.175404309, 0.539268858, 0.145380025, 0.760563606, 0.384400483,
           0.70664456, 0.042186345, 0.36935306, 0.886768254, 0.438805782,
           0.766113852, 0.476812718, 0.687151746, 0.482864162, 0.939177907,
           0.217239545, 0.241372256, 0.751795483, 0.156783283, 0.161982175,
           0.335419826, 0.276520486, 0.206138728, 0.530530843, 0.453191996,
           0.851637817, 0.116777469, 0.425825867, 0.709366443, 0.735309975],

    "x4": [0.701694369, 0.508783439, 0.494932421, 0.613023557, 0.287076101,
           0.195102841, 0.441324251, 0.206566281, 0.24384229, 0.778857694,
           0.776209905, 0.133365734, 0.081516564, 0.082569231, 0.376191726,
           0.959800985, 0.770062476, 0.723827439, 0.856534801, 0.25392378,
           0.999482561, 0.531231498, 0.097796831, 0.096301623, 0.105887338,
           0.552196286, 0.79371778, 0.249007415, 0.503970014, 0.808092013],

    "y": [-22.10828779, -14.60139663, -11.69993246, -16.05376511, -10.06963343,
          -15.48708254, -12.68168498, -16.02639977, -17.04923465, -12.74176599,
          -27.31639636, -13.52764887, -16.6791152, -16.50715856, -17.81799934,
          -26.56182083, -12.75832422, -19.44155762, -28.90327367, -13.70274694,
          -29.4270914, -11.56574199, -26.85778644, -7.966775351, -6.702089255,
          -32.62566022, -19.98949793, -4.025542282, -13.12278233, -23.1394284]
})
new_data = [
    (0.400067, 0.398817, 0.329683, 0.329683, -0.0550098592850463),  # week 1
    (0.455200, 0.430706, 0.202292, 0.483927, -4.12374978864314),  # week 2
    (0.364181, 0.383684, 0.454465, 0.439950, -0.258613723899898),  # week 3
    (0.416724, 0.412932, 0.389241, 0.408404, 0.575635082133228),  # week 4
    (0.354823, 0.445703, 0.432119, 0.363884, 0.255414473109301),  # week 5
    (0.405765, 0.426951, 0.437980, 0.395454, 0.357283309977458),  # week 6
    (0.410587, 0.385222, 0.363456, 0.408410, 0.543874291018458),  # week 7
    (0.369289, 0.393042, 0.361612, 0.429093, 0.536091965697352),  # week 8
    (0.342062, 0.413084, 0.402219, 0.388798, 0.361704198337915),  # week 9
    (0.368698, 0.483077, 0.419323, 0.435646, -0.82579127316184),  # week 10
    (0.425335, 0.397672, 0.412615, 0.380616, 0.486092738670525),  # week 11
    (0.404533, 0.430322, 0.382219, 0.393035, 0.293999511147515),  # week 12
    (0.352036, 0.431533, 0.399742, 0.346596, 0.484137828187819),  # week 13
]
df_new = pd.DataFrame(new_data, columns=["x1", "x2", "x3", "x4", "y"])
df_all = pd.concat([df_init, df_new], ignore_index=True)
# Extract input (X) and output (y)
X_check = df_all[["x1", "x2", "x3", "x4"]].values  # shape (30, 4)
y_check = df_all["y"].values.reshape(-1, 1)  # shape (30, 1)

print("Dataset shape:", X_check.shape, y_check.shape)
print(df_all.tail())

# For later use in model training
X_init = df_all[["x1", "x2", "x3", "x4"]].to_numpy()
y_raw = df_all["y"].to_numpy()

Dataset shape: (43, 4) (43, 1)
          x1        x2        x3        x4         y
38  0.342062  0.413084  0.402219  0.388798  0.361704
39  0.368698  0.483077  0.419323  0.435646 -0.825791
40  0.425335  0.397672  0.412615  0.380616  0.486093
41  0.404533  0.430322  0.382219  0.393035  0.294000
42  0.352036  0.431533  0.399742  0.346596  0.484138


## Optimisation Model

In [3]:
# --- Adjustable parameters ---
n_candidates = 15000       # number of random candidate points to explore
nu = 2.5                  # smoothness parameter for Matern kernel
noise_level = 1.0         # assumed noise (for WhiteKernel)
length_scale = 0.3        # initial length scale for Matern
beta = 1.0               # exploration parameter for UCB (higher = more exploration)
random_state = 42         # reproducibility

# --- Define kernel and GP model ---
kernel = ConstantKernel(1.0, (1e-2, 1e2)) * Matern(length_scale=length_scale, nu=nu) + WhiteKernel(noise_level=noise_level)
gp = GaussianProcessRegressor(kernel=kernel, normalize_y=True, random_state=random_state)

# --- Fit GP to initial data ---
gp.fit(X_init, y_raw)

# --- Generate candidate points uniformly in [0,1]^4 ---
X_candidates = np.random.rand(n_candidates, 4)

# --- Predict mean and std for each candidate ---
mean, std = gp.predict(X_candidates, return_std=True)

# --- Convert mean/std back to original y scale ---
mean_orig = mean * np.std(y_raw) + np.mean(y_raw)
std_orig = std * np.std(y_raw)

# --- Compute UCB acquisition function (in original scale) ---
ucb = mean_orig + beta * std_orig   # for maximization

# --- Get top 5 candidates ---
top_idx = np.argsort(ucb)[-5:][::-1]
top_candidates = X_candidates[top_idx]
top_ucb_values = ucb[top_idx]
top_pred_y = mean_orig[top_idx]

# --- Display results ---
df_top = pd.DataFrame(top_candidates, columns=["x1", "x2", "x3", "x4"])
df_top["Pred_y"] = top_pred_y
df_top["UCB_value"] = top_ucb_values

print("\nTop 5 candidate points (highest UCB):")
print(df_top)
print("\nBest guess (highest UCB):")
print(df_top.iloc[0])


Top 5 candidate points (highest UCB):
         x1        x2        x3        x4     Pred_y  UCB_value
0  0.358676  0.336507  0.395649  0.410727  -8.156154  -5.809147
1  0.391318  0.337859  0.389085  0.317188 -10.536053  -7.463144
2  0.389391  0.431865  0.451007  0.366238 -10.070559  -8.244447
3  0.448182  0.323527  0.382253  0.354845 -11.580425  -8.371764
4  0.461185  0.388719  0.354744  0.378986 -11.605560  -9.503234

Best guess (highest UCB):
x1           0.358676
x2           0.336507
x3           0.395649
x4           0.410727
Pred_y      -8.156154
UCB_value   -5.809147
Name: 0, dtype: float64
